```sql
-- COPYRIGHT (C) 2022 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.
```

# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Flow Diagrams](#flow-diagrams)**
2. **[Dependencies](#dependencies)**
3. **[Input Layer](#input-layer)**
4. **[Intermediate Layer](#intermediate-layer)**
5. **[Ouput Layer](#output-layer)**


# **FLOW DIAGRAMS**

---

[![](https://mermaid.ink/img/pako:eNqNkU1rwzAMhv-K0SmFJiO7LYdd1h0CWQMt9JKUoMVK661Rgu10lNL_Pq1tYB8U5pOQXj3SKx-h7jRBAs2u-6i3aL1S2aJkJc8NrxuL_Val3A9eZXgge6ksbVAs0tXLeqLCUJXwlK_SWRg_VHu0BtkTR7XblyDVR7UsAudxY3gzWV_aifWfCZ5sS9qgpx-DzoS0CMw3wW1MPvhfm6ZnQB4XwXU1d7t7nkVvruNLNo9VGIm1BpMGQ9cbZmGCigS3mmdxUIzA9eRfHfdBwSgOKm2ahixxTXcjoqp6sjWxnIlGnOwHU2jFNRotH3T8Spfgt9RSCYmEGu17CSWfRDf0Wi7zrI3vLMj8naMp4OC75YFrSLwdaBTNDIrf9qo6fQI2SqUu)](https://mermaid-js.github.io/mermaid-live-editor/edit#pako:eNqNkU1rwzAMhv-K0SmFJiO7LYdd1h0CWQMt9JKUoMVK661Rgu10lNL_Pq1tYB8U5pOQXj3SKx-h7jRBAs2u-6i3aL1S2aJkJc8NrxuL_Val3A9eZXgge6ksbVAs0tXLeqLCUJXwlK_SWRg_VHu0BtkTR7XblyDVR7UsAudxY3gzWV_aifWfCZ5sS9qgpx-DzoS0CMw3wW1MPvhfm6ZnQB4XwXU1d7t7nkVvruNLNo9VGIm1BpMGQ9cbZmGCigS3mmdxUIzA9eRfHfdBwSgOKm2ahixxTXcjoqp6sjWxnIlGnOwHU2jFNRotH3T8Spfgt9RSCYmEGu17CSWfRDf0Wi7zrI3vLMj8naMp4OC75YFrSLwdaBTNDIrf9qo6fQI2SqUu)

Required steps:

1. **[Source](https://data.rivm.nl/covid-19/COVID-19_varianten.csv)**: contains the `"virus variants"` data collected by RIVM that will be used for further processing.
2. **[Staging](#input-layer)**: inserts the raw `Source` data within the `[VWSSTAGE].[RIVM_VARIANTS_NL]` table.
3. **[Intermediate](#intermediate-layer)**: inserts the `Staging` data and cast the correct `Database Types` within `[VWSINTER].[RIVM_VARIANTS_NL]` using business logic implemented in `[dbo].[SP_INSERT_IL_RIVM_VARIANTS_NL]`.
4. **[Variants](#output-layer)**: processes the `Intermediate` data to require the ***variants*** and ***variant percentage differences***, and inserts the results within `[VWSDEST].[RIVM_VARIANTS_NL]` using business logic implemented in `[dbo].[SP_INSERT_OL_RIVM_VARIANTS_NL]`.
   - The `[VWSDEST].[V_RIVM_VARIANTS_NL]` and `[VWSDEST].[V_RIVM_VARIANTS_DIFFERENCES_NL]` are used to generate the json-objects `variants` and `named_difference/variants__percentage`, respectively, within the `NL.json`.

- **<font color=teal>IL</font>**: Intermediate Layer
- **<font color=teal>OL</font>**: Output Layer

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/DataFactory/Utils/Functions.ipynb",
        "src/DataFactory/Utils/Schemas.ipynb",
        "src/DataFactory/Utils/Protos.ipynb",
        "src/DataFactory/Statics/pl_ProcessSubvariantMapping.ipynb"
        // Additional dependencies (!NOTE! DO NOT FORGET THE COMMA (i.e. ,))
    ]
}
```

# **INPUT LAYER**

---

## **<span style='color:teal'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSSTAGE].[RIVM_VARIANTS_NL]'))
CREATE TABLE [VWSSTAGE].[RIVM_VARIANTS_NL] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[VERSION] [VARCHAR](100) NULL,
	[DATE_OF_REPORT] [VARCHAR](100) NULL,
	[DATE_OF_STATISTICS_WEEK_START] [VARCHAR](100) NULL,
	[VARIANT_CODE] [VARCHAR](100) NULL,
	[VARIANT_NAME] [VARCHAR](100) NULL,
	[ECDC_CATEGORY] [VARCHAR](100) NULL,
	[SAMPLE_SIZE] [VARCHAR](100) NULL,
	[VARIANT_CASES] [VARCHAR](100) NULL
);
GO

# **INTERMEDIATE LAYER**

---

## **<span style='color:teal'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSINTER].[RIVM_VARIANTS_NL]'))
CREATE TABLE [VWSINTER].[RIVM_VARIANTS_NL] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[VERSION] [INT] NULL,
	[DATE_OF_REPORT] [DATETIME] NULL,
	[DATE_OF_STATISTICS_WEEK_START] [DATETIME] NULL,
	[VARIANT_CODE] [VARCHAR](100) NULL,
	[VARIANT_NAME] [VARCHAR](100) NULL,
	[ECDC_CATEGORY] [VARCHAR](100) NULL,
	[SAMPLE_SIZE] [INT] NULL,
	[VARIANT_CASES] [INT] NULL
);
GO

## **<span style='color:teal'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): STAGING TO INTERMEDIATE.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_IL_RIVM_VARIANTS_NL]
AS
BEGIN
    INSERT INTO [VWSINTER].[RIVM_VARIANTS_NL] (
        [DATE_OF_REPORT],
        [DATE_OF_STATISTICS_WEEK_START],
        [VARIANT_CODE],
        [VARIANT_NAME],
        [ECDC_CATEGORY],
        [SAMPLE_SIZE],
        [VARIANT_CASES]
    )
    SELECT
        [dbo].[TRY_CONVERT_TO_DATETIME]([DATE_OF_REPORT]),
        [dbo].[TRY_CONVERT_TO_DATETIME]([DATE_OF_STATISTICS_WEEK_START]),
        [VARIANT_CODE],
        [VARIANT_NAME],
        [ECDC_CATEGORY],
        CAST([SAMPLE_SIZE] AS INT),
        CAST([VARIANT_CASES] AS INT)
    FROM
        [VWSSTAGE].[RIVM_VARIANTS_NL]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[RIVM_VARIANTS_NL])
END;
GO

# **OUTPUT LAYER**

---

## **<span style='color:teal'>VARIANTS NL</span>**

### **<span style='color:cadetblue'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSDEST].[RIVM_VARIANTS_NL]'))
BEGIN
CREATE TABLE [VWSDEST].[RIVM_VARIANTS_NL] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED]  [DATETIME] DEFAULT GETDATE(),
	[DATE_OF_REPORT] [DATETIME] NULL,
	[DATE_OF_STATISTICS_WEEK_START] [DATETIME] NULL,
	[VARIANT_CODE] [VARCHAR](100) NULL,
	[VARIANT_NAME] [VARCHAR](100) NULL,
	[VARIANT_LABEL] [VARCHAR](100) NULL,
	[IS_VARIANT_OF_CONCERN] [VARCHAR](100) NULL,
	[SAMPLE_SIZE] [INT] NULL,
	[VARIANT_CASES] [INT] NULL,
	[VARIANT_PERCENTAGE] [DECIMAL](16, 3) NULL,
	[HAS_HISTORICAL_SIGNIFICANCE] [VARCHAR](100) NULL
);
END
ELSE
BEGIN
	IF NOT EXISTS(SELECT * FROM SYS.COLUMNS 
        WHERE [NAME] IN (N'VARIANT_LABEL')
        AND [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[RIVM_VARIANTS_NL]'))
	BEGIN
		ALTER TABLE [VWSDEST].[RIVM_VARIANTS_NL] 
		ADD 
			[VARIANT_LABEL] VARCHAR (100)
	END
END
GO

-- 2) CREATE INDEX(S).....
DROP INDEX IF EXISTS [NCIX_DLI_RIVM_VARIANTS_NL] 
	ON [VWSDEST].[RIVM_VARIANTS_NL]
GO

CREATE NONCLUSTERED INDEX [NCIX_DLI_RIVM_VARIANTS_NL]
    ON [VWSDEST].[RIVM_VARIANTS_NL] (
		[DATE_LAST_INSERTED]
	)
GO

### **<span style='color:cadetblue'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): INTERMEDIATE TO DESTINATION.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_OL_RIVM_VARIANTS_NL]
AS
BEGIN
     WITH SUBVARIANTS AS (
          SELECT 
               [VARIANT_CODE],
               [VARIANT_NAME],
               [LABEL] AS [VARIANT_LABEL],
               [HISTORICALY_SIGNIFICANT]
          FROM [VWSSTATIC].[RIVM_SUBVARIANT_MAPPING]  WITH(NOLOCK)
          WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[MASTERDATA_VARIANTS] WITH(NOLOCK))
               AND [HISTORICALY_SIGNIFICANT] = 1
     ), 
     VARIANTS AS (
          SELECT
               [DATE_OF_REPORT],
               [DATE_OF_STATISTICS_WEEK_START],
               CASE T1.[VARIANT_CODE]
                    WHEN 'BA.4' THEN 'BA.4/5'
                    WHEN 'BA.5' THEN 'BA.4/5'
                    ELSE T1.[VARIANT_CODE]
               END AS [VARIANT_CODE],
               CASE 
                    WHEN T1.[VARIANT_NAME] = '' THEN T2.[VARIANT_NAME]
                    ELSE T1.[VARIANT_NAME]
               END AS [VARIANT_NAME],
               T2.[VARIANT_LABEL],
               [ECDC_CATEGORY],
               IIF([ECDC_CATEGORY] = 'VOC' 
                    OR (CASE T1.[VARIANT_CODE]
                         WHEN 'BA.4' THEN 'BA.4/5'
                         WHEN 'BA.5' THEN 'BA.4/5'
                         ELSE T1.[VARIANT_CODE]
                    END) = 'B.1.1.529'
                    OR (CASE 
                         WHEN T1.[VARIANT_NAME] = '' THEN LOWER(T2.[VARIANT_NAME])
                         ELSE LOWER(T1.[VARIANT_NAME])
                    END) IN ('alpha', 'beta','gamma'), 'true','false') AS [IS_VARIANT_OF_CONCERN],
               CASE T2.[HISTORICALY_SIGNIFICANT]
                    WHEN 1 THEN 'true'
                    ELSE IIF(
                         CASE 
                              WHEN T1.[VARIANT_NAME] = '' THEN LOWER(T2.[VARIANT_NAME])
                              ELSE LOWER(T1.[VARIANT_NAME])
                         END IN ('alpha','beta','gamma'), 'true', 'false'
                    )
               END AS [HAS_HISTORICAL_SIGNIFICANCE],
               [SAMPLE_SIZE],
               [VARIANT_CASES]
               -- CAST([VARIANT_CASES] AS FLOAT) / [SAMPLE_SIZE] AS [VARIANT_PERCENTAGE],
               -- [DATE_LAST_INSERTED]
          FROM 
               [VWSINTER].[RIVM_VARIANTS_NL] T1
          INNER JOIN SUBVARIANTS T2 ON LOWER(T2.[VARIANT_CODE]) = LOWER(T1.[VARIANT_CODE])
          WHERE [DATE_LAST_INSERTED] = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSINTER].[RIVM_VARIANTS_NL]) 
               AND ([ECDC_CATEGORY] IN ('VOC', 'VOI') OR T1.[VARIANT_NAME] IN (SELECT [VARIANT_NAME] FROM SUBVARIANTS))
     ),
     BASE_CTE AS (
          SELECT 
               [DATE_OF_REPORT], 
               [DATE_OF_STATISTICS_WEEK_START], 
               [VARIANT_CODE], 
               [VARIANT_NAME], 
               [VARIANT_LABEL], 
               [ECDC_CATEGORY], 
               [IS_VARIANT_OF_CONCERN], 
               [HAS_HISTORICAL_SIGNIFICANCE], 
               [SAMPLE_SIZE], 
               SUM([VARIANT_CASES]) AS [VARIANT_CASES],
               CAST(SUM([VARIANT_CASES]) AS FLOAT) / [SAMPLE_SIZE] AS [VARIANT_PERCENTAGE]
          FROM 
               VARIANTS
          GROUP BY 
               [DATE_OF_REPORT], 
               [DATE_OF_STATISTICS_WEEK_START], 
               [VARIANT_CODE], 
               [VARIANT_NAME], 
               [VARIANT_LABEL], 
               [ECDC_CATEGORY], 
               [IS_VARIANT_OF_CONCERN], 
               [HAS_HISTORICAL_SIGNIFICANCE], 
               [SAMPLE_SIZE]
     ),
     -- USE A GRID WITH ALL VALID DATE AND VARIANT COMBINATIONS
     -- TO COVER THE SITUATION THAT A VARIANT IS NOT PRESENT IN ALL WEEKS
     GRID_CTE AS (
          SELECT DISTINCT
               T1.[DATE_OF_STATISTICS_WEEK_START],
               T2.[VARIANT_CODE]
          FROM 
               BASE_CTE AS T1
          CROSS JOIN 
               (SELECT DISTINCT [VARIANT_CODE] FROM BASE_CTE) AS T2
     ),
     -- GRID IS FILLED WITH ALL PRESENT DATA
     FULL_CTE AS (
          SELECT
               T0.[DATE_OF_STATISTICS_WEEK_START],
               T0.[VARIANT_CODE],
               T1.[DATE_OF_REPORT],
               T1.[VARIANT_NAME],
               T1.[VARIANT_LABEL],
               T1.[ECDC_CATEGORY],
               T1.[IS_VARIANT_OF_CONCERN],
               T1.[HAS_HISTORICAL_SIGNIFICANCE],
               T1.[SAMPLE_SIZE],
               T1.[VARIANT_CASES],
               T1.[VARIANT_PERCENTAGE]
          FROM 
               GRID_CTE AS T0
          LEFT JOIN BASE_CTE AS T1 ON T0.[DATE_OF_STATISTICS_WEEK_START] = T1.[DATE_OF_STATISTICS_WEEK_START] 
               AND T0.[VARIANT_CODE] = T1.[VARIANT_CODE]
     ),
     TOTALS_CTE AS (
          SELECT
               [DATE_OF_REPORT],
               [DATE_OF_STATISTICS_WEEK_START],
               MAX([SAMPLE_SIZE]) AS [SAMPLE_SIZE],
               SUM([VARIANT_CASES]) AS [VARIANT_CASES]
          FROM 
               FULL_CTE
          WHERE [ECDC_CATEGORY] IN ('VOC', 'VOI') OR [VARIANT_CODE] = 'B.1.1.529'
          GROUP BY 
               [DATE_OF_REPORT], 
               [DATE_OF_STATISTICS_WEEK_START]
     ),
     TOTALS_CTE2 AS (
          SELECT
               [DATE_OF_REPORT],
               [DATE_OF_STATISTICS_WEEK_START],
               MAX([SAMPLE_SIZE]) AS [SAMPLE_SIZE],
               SUM([VARIANT_CASES]) AS [VARIANT_CASES]
          FROM FULL_CTE
          WHERE [ECDC_CATEGORY] IN ('VOC') OR [HAS_HISTORICAL_SIGNIFICANCE] = 'true' OR [VARIANT_CODE] = 'B.1.1.529'
          GROUP BY 
               [DATE_OF_REPORT], 
               [DATE_OF_STATISTICS_WEEK_START]
     )
     INSERT INTO [VWSDEST].[RIVM_VARIANTS_NL] (
          [DATE_OF_REPORT],
          [DATE_OF_STATISTICS_WEEK_START],
          [VARIANT_CODE],
          [VARIANT_NAME],
          [VARIANT_LABEL],
          [IS_VARIANT_OF_CONCERN],
          [HAS_HISTORICAL_SIGNIFICANCE],
          [SAMPLE_SIZE],
          [VARIANT_CASES],
          [VARIANT_PERCENTAGE]
     )
     SELECT
          [DATE_OF_REPORT],
          [DATE_OF_STATISTICS_WEEK_START],
          [VARIANT_CODE],
          [VARIANT_NAME],
          [VARIANT_LABEL],
          [IS_VARIANT_OF_CONCERN],
          [HAS_HISTORICAL_SIGNIFICANCE],
          [SAMPLE_SIZE],
          [VARIANT_CASES],
          ROUND(CAST([VARIANT_CASES]*100 AS FLOAT) / [SAMPLE_SIZE],1) AS [VARIANT_PERCENTAGE]
     FROM 
          FULL_CTE
     UNION ALL
     --OTHER VARIANTS TABEL
     SELECT
          [DATE_OF_REPORT],
          [DATE_OF_STATISTICS_WEEK_START],
          '' AS [VARIANT_CODE],
          'Other_Table' AS [VARIANT_NAME],
          'Other_Table' AS [VARIANT_LABEL],
          'false' AS [IS_VARIANT_OF_CONCERN],
          'false' AS [HAS_HISTORICAL_SIGNIFICANCE],
          [SAMPLE_SIZE],
          [SAMPLE_SIZE] - [VARIANT_CASES] AS [VARIANT_CASES],
          ROUND((CAST(([SAMPLE_SIZE] - [VARIANT_CASES]) AS FLOAT)*100) / [SAMPLE_SIZE],1) AS [VARIANT_PERCENTAGE]
     FROM 
          TOTALS_CTE
     UNION ALL
     --OTHER VARIANTS GRAFIEK
     SELECT
          [DATE_OF_REPORT],
          [DATE_OF_STATISTICS_WEEK_START],
          '' AS [VARIANT_CODE],
          'Other_Graph' AS [VARIANT_NAME],
          'Other_Graph' AS [VARIANT_LABEL],
          'false' AS [IS_VARIANT_OF_CONCERN],
          'true' AS [HAS_HISTORICAL_SIGNIFICANCE],
          [SAMPLE_SIZE],
          [SAMPLE_SIZE] - [VARIANT_CASES] AS [VARIANT_CASES],
          ROUND((CAST(([SAMPLE_SIZE] - [VARIANT_CASES]) AS FLOAT)*100) / [SAMPLE_SIZE],1) AS [VARIANT_PERCENTAGE]
     FROM 
          TOTALS_CTE2
END;
GO

### **<span style='color:cadetblue'>VIEWS</span>**

In [ ]:
-- 1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSDEST].[V_RIVM_VARIANTS_NL] AS
WITH CTE AS (
    SELECT
        -- [DATE_OF_REPORT]
        -- [VARIANT_CODE]
        [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_OF_STATISTICS_WEEK_START]) AS [DATE_START_UNIX],
        [dbo].[CONVERT_DATETIME_TO_UNIX]([dbo].[WEEK_END]([DATE_OF_STATISTICS_WEEK_START])) AS [DATE_END_UNIX],
        LOWER([VARIANT_NAME]) AS [VARIANT_NAME],
        [IS_VARIANT_OF_CONCERN],
        [HAS_HISTORICAL_SIGNIFICANCE],
        [SAMPLE_SIZE],
        [VARIANT_CASES] AS [OCCURRENCE],
        [VARIANT_PERCENTAGE] AS [PERCENTAGE],
        [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_LAST_INSERTED]) AS [DATE_OF_INSERTION_UNIX]
    FROM [VWSDEST].[RIVM_VARIANTS_NL] WITH(NOLOCK)
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[RIVM_VARIANTS_NL] WITH(NOLOCK))
)
SELECT
    [DATE_START_UNIX],
    [DATE_END_UNIX],
    [VARIANT_NAME],
    [IS_VARIANT_OF_CONCERN],
    [HAS_HISTORICAL_SIGNIFICANCE],
    [SAMPLE_SIZE],
    [OCCURRENCE],
    [PERCENTAGE],
    [DATE_OF_INSERTION_UNIX]
FROM CTE
GO

### **<span style='color:cadetblue'>VIEWS | CONFIGURATION</span>**

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_RIVM_VARIANTS_NL',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'variants',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = 'VARIANT_NAME',
        @grouped_last_update_name VARCHAR(50) = 'DATE_START_UNIX',
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = 'DATE_START_UNIX|DATE_END_UNIX|IS_VARIANT_OF_CONCERN|HAS_HISTORICAL_SIGNIFICANCE|SAMPLE_SIZE|OCCURRENCE|PERCENTAGE|DATE_OF_INSERTION_UNIX',
        @layout_type_id INT = 1, -- LASTVALUES
        @last_update_name VARCHAR(50) = 'DATE_START_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

## **<span style='color:teal'>VARIANTS DIFFERENCES NL</span>**

### **<span style='color:cadetblue'>VIEWS</span>**

In [ ]:
-- 1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSDEST].[V_RIVM_VARIANTS_DIFFERENCES_NL] AS
WITH CTE AS (
    SELECT
        [DATE_START_UNIX],
        LAG([DATE_START_UNIX],1) OVER (PARTITION BY [VARIANT_NAME] ORDER BY [DATE_START_UNIX]) AS [OLD_DATE_UNIX],
        [PERCENTAGE] AS [NEW_VALUE],
        [VARIANT_NAME] AS [NAME],
        LAG([PERCENTAGE],1) OVER (PARTITION BY [VARIANT_NAME] ORDER BY [DATE_START_UNIX]) AS [OLD_VALUE],
        ROW_NUMBER() OVER (PARTITION BY [VARIANT_NAME] ORDER BY [DATE_START_UNIX] DESC) AS [RANK_DATE]
    FROM [VWSDEST].[V_RIVM_VARIANTS_NL]
    WHERE [HAS_HISTORICAL_SIGNIFICANCE] = 'false'
)
SELECT
    [NAME],
    [OLD_VALUE],
    -- NEW_VALUE
    ROUND([NEW_VALUE] - [OLD_VALUE], 1) AS [DIFFERENCE],
    [DATE_START_UNIX] AS [NEW_DATE_UNIX],
    [OLD_DATE_UNIX] as [OLD_DATE_UNIX]
FROM CTE
WHERE [RANK_DATE] = 1
GO

### **<span style='color:cadetblue'>VIEWS | CONFIGURATION</span>**

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_RIVM_VARIANTS_DIFFERENCES_NL',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'named_difference/variants__percentage',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = NULL,
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = 'NAME|OLD_VALUE|DIFFERENCE|OLD_DATE_UNIX|NEW_DATE_UNIX',
        @layout_type_id INT = 4, -- LASTVALUES
        @last_update_name VARCHAR(50) = 'NEW_DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO